In [8]:
from functools import wraps
import datetime
def loger(s):
    def log(fn):
        @wraps(fn)
        def wrap(*args,**kwargs):
            start = datetime.datetime.now()
            ret = fn(*args,**kwargs)
            end = datetime.datetime.now()
            print(s)
            return ret
        return wrap
    return log
        

In [19]:
d = lambda x:x+3
import time
@loger(s=d(3))
def sleep(x):
    time.sleep(x)

In [20]:
sleep(1)

6


In [21]:
def loger(s):
    def _logger(p=lambda name,t :print('function: {},took: {}'.format(name,t))):
        def __logger(fn):
            @wraps(fn)
            def wrap(*args,**kwargs):
                start = datetime.datetime.now()
                ret = fn(*args,**kwargs)
                end = datetime.datetime.now()
                if (end - start).total_seconds() >s:
                    p(fn.__name__,end-start)
                return ret
            return wrap
        return __logger
    return _logger

In [22]:
logger = loger(2)
@logger()
def sleep(x):
    time.sleep(x)

In [23]:
sleep(3)

function: sleep,took: 0:00:03.004215


In [3]:
sleep(3)

functions sleep is use 0:00:03.003141


In [148]:
from functools import wraps
import datetime
def cache(timeout=False):
    def cached(fn):
        cache_dic = {fn.__name__:dict.fromkeys(('args','kwargs','ret','timelive'))}
        @wraps(fn)
        def wrap(*args,**kwargs):
            if timeout and cache_dic[fn.__name__]['timelive']:
                now = datetime.datetime.now().timestamp() - cache_dic[fn.__name__]['timelive']
                print(now)
                if now >= timeout:
                    cache_dic[fn.__name__]=dict.fromkeys(('args','kwargs','ret','timelive'))
                    
            if cache_dic[fn.__name__]['args'] != args or cache_dic[fn.__name__]['kwargs'] != kwargs:
                start = datetime.datetime.now().timestamp()
                cache_dic[fn.__name__]['args'] = args
                cache_dic[fn.__name__]['kwargs'] = kwargs
                cache_dic[fn.__name__]['ret'] = fn(*args,**kwargs)
                cache_dic[fn.__name__]['timelive'] = start
            return cache_dic[fn.__name__]['ret']
        return wrap
    return cached

In [152]:
import time
@cache(timeout=20)
def add(x,y):
    time.sleep(x)
    return x +y

In [153]:
add(3,5)

8

In [167]:
add(3,5)

4.68508505821228


8

In [6]:
import datetime
import functools
def logger(s):
    def _logger(fn):
        #@copy_proprities(fn)
        #@functools.wraps(fn)
        def warp(*args,**kwargs):
            start = datetime.datetime.now()
            ret = fn(*args,**kwargs)
            end = datetime.datetime.now()
            if (end-start).total_seconds() > s:
                print('call {} took {}'.format(fn.__name__,end - start))
            return ret
        return warp
    return _logger


In [7]:
import time
@logger(1)
def sleep(x):
    time.sleep(x)

In [8]:
a = sleep(4)

call sleep took 0:00:04.005616


In [9]:
sleep.__name__

'warp'

In [70]:
class rule:
    dic = {}
    def ins(self,name):
        def _ins(fn):
            self.dic[name] = fn
            return fn
        return _ins
    def default(self):
        print('default command')
    def run(self):
        while True:
            cmd = input(">>>")
            if cmd.lower() == 'quit':
                return
            self.dic.get(cmd,self.default)()

In [71]:
rule = rule()

In [72]:
@rule.ins('1')
def b():
    print('abc')

In [73]:
rule.dic

{'1': <function __main__.b>}

In [74]:
rule.run()

>>>1
abc
>>>
default command
>>>
default command
>>>
default command
>>>2
default command
>>>3
default command
>>>1
abc
>>>quit


In [18]:
def b64encode(data: bytes) -> str:
    table = b'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'
    encoded = bytearray()
    c = 0
    for x in range(3, len(data)+1, 3):
        print(data[c:x])
        i = int.from_bytes(data[c: x], 'big')
        for j in range(1, 5):
            encoded.append(table[i >> (24 - j*6) & 0x3f])
        c += 3
    r = len(data) - c
    print(encoded)
    if r > 0:
        i = int.from_bytes(data[c:], 'big') << (3-r) * 8
        print(i)
        for j in range(1, 5-(3-r)):
            encoded.append(table[i >> (24 - j*6) & 0x3f])
        for _ in range(3-r):
            encoded.append(int.from_bytes(b'=', 'big'))
    return encoded.decode()

In [99]:
def re_convert(x):    #Base64编码转成字符串 方法
    rule = b'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'
    data = x.encode()
    #(((24<<18)+(22<<12)+(9<<6)+35)<<24) + ((25<<18)+(0<<12)) >> (8*2)
    tmp = 0
    flag = 0
    for j in range(4,len(data)+1,4):
        for i,d in enumerate(data[j-4:j]):
            if d in rule:
                tmp += ((rule.index(d))<<(24-((i+1)*6)))
            else:
                flag +=1
        if len(data) >4:
            tmp = tmp<<24
    if flag:
        tmp >>(8*flag)
    return tmp.to_bytes(len(data)+1,'big').strip(b'\x00').decode()      

In [102]:
re_convert('YWFkZmFzZGNh')

'aadfasdca'

In [59]:
import base64

In [101]:
base64.b64encode(b'aadfasdca')

b'YWFkZmFzZGNh'

In [24]:
rule = b'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789+/'

In [75]:
rule.index(b'E')

4

In [76]:
bin(4)

'0b100'

In [1]:
int('11000000000000000000000',base=2)

6291456

In [2]:
24<<18

6291456

In [4]:
int('10110000000000000',base=2)

90112

In [3]:
22 << 12

90112

In [40]:
int.from_bytes(b'abcd','big')

1633837924

In [ ]:
24 22 9  35  25 0
24 22 9 35 25 6 4 0

In [68]:
((24<<24)+(22<<18)+(9<<12)+(35<<6)+25)<<2

1633837924

In [69]:
(24<<30) + (22<<24)+(9<<18)+(35<<12)+(25<<6)+0

26141406784

In [ ]:
110000 00000000000000000   24<<18
       10110000000000000   22<<12
110000 10110000000000000   (24<<18)+(22<<12)

             1001000000
11000010110001001000000
                 100011
11000010110001001100011



YWJjZ                         A
((24<<24)+(22<<18)+(9<<12)+(35<<6)+25)<<2
1100001011000100110001101100100  0000 000000 000000
1100001011000100110001101100100 00000000   00 00 00 00

11000010110001001100011011001  


YWJjZ                           G      E
(24<<36)+(22<<30)+(9<<24)+(35<<18)+(25<<12)+(6<<6)+(4)
                              000110 000100    000000
11000010110001001100011011001 000110 0001    00000000
11000010110001001100011011001 000110 0001

In [ ]:
24 22 9 35    25 0 0 0
24 22 9 35    25 6 4 0
YWJj ZGE=
YWJjZ A==

In [71]:
int.from_bytes(b'abcda','big')

418262508641

In [72]:
bin(418262508641)

'0b110000101100010011000110110010001100001'

In [88]:
(((24<<18)+(22<<12)+(9<<6)+35)<<24) + ((25<<18)+(0<<12)) >> (8*2)

1633837924

In [106]:
(1010).to_bytes(2,'big')

b'\x03\xf2'

In [95]:
from functools import wraps
import inspect
import datetime

def cache(exp=0,maxsize=5):
    def _cache(fn): ## fn cache
        cache = {}  #定义缓存的字典
        @wraps(fn)
        def wrap(*args, **kwargs):
            # TODO key 如何拼装
            key = [] #用来装所有的形参
            names = set() 
            params = inspect.signature(fn).parameters #列出一个形参的字典
            #实参
            for i, arg in enumerate(args): #遍历实参的下标和参数值
                name = list(params.keys())[i] #name赋值为对应位置的形参
                key.append((name, arg)) #在空列表key中添加一个元组（形参和对应的实参）
                names.add(name) #在空集合names中添加形参name
            key.extend(kwargs.items()) #kwargs转换成元组，扩展到列表key中
            names.update(kwargs.keys()) #将kwargs中所有的key更新到集合names中
            #形参
            for k, v in params.items(): #遍历形参字典
                if k not in names: #判断key不在集合names中
                    key.append((k, v.default)) #将key和形参默认值
            key.sort(key=lambda x: x[0]) #对列表key中所有的元素进行排序
            key = '&'.join(['{}={}'.format(name, arg) for name, arg in key]) #拼接所有形参实参对应关系形成一个缓存的key
            if len(cache.keys()) >= maxsize:
                print(min(cache,key=lambda k:cache[k][2]))
                cache.pop(min(cache,key=lambda k:cache[k][2]),None)
            if key in cache.keys(): #判断key在缓存字典中
                cache[key][2] +=1
                ret, timestamp,hotkey = cache[key] #解构字典的缓存value
                if exp == 0 or datetime.datetime.now().timestamp() - timestamp < exp: #判断value中时间戳是否超时
                    print(cache)
                    return ret
            ret = fn(*args, **kwargs) #第一次访问时ret等于函数的执行结果
            hotkey=0
            cache[key] = [ret, datetime.datetime.now().timestamp(),hotkey] #在字典cache中存入key和value
            print(cache)
            return ret
        return wrap
    return _cache

In [84]:
@cache(maxsize=3)
def add(x,y):
    return x+y

In [ ]:
def command():      #用函数来封装闭包操作
    commands = {}   #用于存储注册的字典

    def register(command):   #注册装饰器，带参数，参数名为key，值为对应的函数名
        def _register(fn):
            if command in commands:    #检查是否已经注册过了。
                raise Exception('command {} exist'.format(command))  #注册过的抛出异常。
            commands[command] = fn
            return fn
        return _register

    def default_fn():   #默认函数，当无注册函数时返回这个函数的调用
        print('unkonwn command')

    def run():           #执行函数
        while True:
            cmd = input('>>')
            if cmd.strip() == 'quit':
                return
            commands.get(cmd.strip(),default_fn)()   #柯里化调用。取值后后面添加()来调用这个函数。
    
    return register, run

In [21]:
@cache()
def b(x,y):
    return x+y

In [23]:
b(3,5)

cache hit
{'x=3&y=5': (8, 1484039561.492666)}


8

In [1]:
d = {'a':1,'b':2,'c':3}

In [3]:
len(d.keys())

3

In [31]:
min(d,key=lambda k:d[k])

'a'